In [5]:
# __author__ = "Yifan Zhou"
# __copyright__ = "Copyright 2021, DOE Project"
# __email__ = "yifan.zhou.1@stonybrook.edu"
import pennylane as qml
from setnoise import setNoise
from pennylane import numpy as np

from qiskit_aer.noise import NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error
#from qiskit.providers.aer.noise import noise
# from qiskit.providers.aer.noise import NoiseModel
# from qiskit.providers.aer.noise import QuantumError, ReadoutError
# from qiskit.providers.aer.noise import pauli_error
# from qiskit.providers.aer.noise import depolarizing_error
# from qiskit.providers.aer.noise import thermal_relaxation_error

In [6]:
def setNoise(p_reset, p_meas, p_gate1):
#     # Example error probabilities
#     p_reset = 0.03   # reset error
#     p_meas = 0.1     # measurement error
#     p_gate1 = 0.001  # 1-qubit gate
#     p_gate2 = 0.01   # 2-qubit gate

    # set to float
    p_reset = float(p_reset)
    p_meas = float(p_meas)
    p_gate1 = float(p_gate1)
    
    # QuantumError objects
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    # Add errors to noise model
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_reset, "reset")
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x90", "u1", "u2", "u3", "U", "id", "x", "y", "z", "h", "s", "sdg",
        "t", "tdg", "r", "rx", "ry", "rz", "p"])
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx", "cy", "cz", "swap", "rxx", "ryy", "rzz",
                                "rzx", "cu1", "cu2", "cu3", "cp"])
    
    return noise_model

In [7]:
# this is the ideal device you previously used
n_qubits = 5
dev_ideal = qml.device("default.qubit", wires=n_qubits) # set up an ideal quantum device for baseline performance evaluation.
# dev_ideal = qml.device("default.qubit", analytic=True, wires=n_qubits) - says analytic parameter doesn't exist

# add gate error to see its impact (you can definitely change the noise level)
# error_Set = [10**i for i in np.arange(-4.5, n-0.9, 0.1)]  # a set of gate errors - n is not defined
error_Set = [10**i for i in np.arange(-4.5, -0.9, 0.1)]  # a set of gate errors
# test QNN under a set of gate errors

for i_Set in range(len(error_Set)-1,-1,-1):
    # set up noise level
    gate_error = error_Set[i_Set]
    noise_model = setNoise(0.03, 0.04413, gate_error)
    # set up noisy device
    dev_noisy = qml.device("qiskit.aer", wires=n_qubits, backend='qasm_simulator', backend_options={"basis_gates": noise_model.basis_gates, "noise_model": noise_model})

    # initialize your qnn and run it using dev_noisy instead of dev_ideal
    #<*test_your_qnn*>
    